# Start Simulation

In [1]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__

'1.11.0+cu113'

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-126c436d-0582-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.905s] [ext: omni.kit.menu.create-1.0.2] startup
[1.906s] [ext: omni.mdl-0.1.0] startup
[1.926s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.926s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.927s] [ext: omni.kit.material.library-1.3.10] startup
[1.930s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.931s] [ext: omni.kit.window.file-1.3.16] startup
[1.933s] [ext: omni.kit.context_menu-1.3.9] startup
[1.937s] [ext: omni.kit.window.property-1.6.3] startup
[1.938s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.948s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.952s] [ext: omni.isaac.version-1.0.0] startup
[1.953s] [ext: omni.kit.property.usd-3.14.9] startup
[1.986s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.990s] [ext: omni.hydra.rtx-0.1.0] startup
[2.000s] [ext: omni.renderer-rtx-0.0.0] startup
[2.000s] [ext: omn

In [5]:
pause

NameError: name 'pause' is not defined

# Init Env

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Config

In [ ]:
from task.config import DATA_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
# scene.add_room()

In [ ]:
env.world.step(render=True)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

In [ ]:
pause

#  Learning

In [ ]:
from learning.network.resnet import ResNetFeatureExtractor
from learning.utils import extract_image_feature_and_save

device = torch.device("cuda")
feature_extractor = ResNetFeatureExtractor(output_layer='avgpool').to(device)

# Trajectory

In [ ]:
# traj config
for traj_id in range(5):

    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
    extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
        feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    

    # trajectory
    for step in range(5):

        env.add_scene_obj(mode = "random")

        x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())

        env.put_last_object((x, y)) 
        env.world.step(render=True)

        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
        extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
            feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))

        ## reset
        env.world.reset()
        env.world.step(render=True)


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    print("record: ", record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

# Replay Buffer

In [6]:
import json
replay = json.load(open("/home/yizhou/Research/DeepArrange/Data/Table/Border/4/scene.json"))

In [7]:
from learning.replay_buffer import ReplayBuffer

In [8]:
buffer = ReplayBuffer(max_size=1000)

In [9]:
buffer.add_scene_sample(replay)

In [10]:
buffer.dataset[0][2].device

device(type='cpu')

In [11]:
batch = buffer.sample_batch()

In [12]:
rewards = batch['rewards']
terminals = batch['terminals']
obs = batch['observations']
actions = batch['actions']
next_obs = batch['next_observations']

obj_features = batch['object_features']

In [13]:
from learning.network.sac import *

In [14]:
policy = Policy()

In [15]:
dist = policy(obs, obj_features)

In [18]:
new_obs_actions, log_pi = dist.rsample_and_logprob()

In [19]:
new_obs_actions, log_pi

(tensor([[-0.5816,  0.5362],
         [-0.7767,  0.4089],
         [ 0.2033,  0.5470],
         [ 0.8587, -0.0724]], grad_fn=<TanhBackward0>),
 tensor([-1.3819, -1.2805, -1.5592, -1.4640], dtype=torch.float64,
        grad_fn=<AddBackward0>))

In [16]:
qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()